In [2]:
import numpy as np
import pandas as pd
from keras.datasets import boston_housing

In [3]:
(train_data, train_target), (test_data, test_labels) = boston_housing.load_data()

57026/57026 [==============================] - 0s 5us/step


In [19]:
mean = train_data.mean(axis=0)
std = train_data.std(axis=0) 

In [20]:
train_data -= mean
train_data /= std

test_data -= mean
test_data /= std

In [25]:
from keras import models
from keras import layers

In [63]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', 
                          input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [64]:
k = 4
num_val_samples = len(train_data)//4
all_mae_history = []
num_epoch = 80

for i in range(4):
    print('processing fold #',i)
    
    val_data = train_data[i *  num_val_samples : (i + 1) * num_val_samples]
    val_targets = train_target[i *  num_val_samples : (i + 1) * num_val_samples]
    
    partial_train = np.concatenate(
        [train_data[:i * num_val_samples],
        train_data[(i + 1) * num_val_samples:]],
                                   axis=0)
    partial_target = np.concatenate([train_target[:i * num_val_samples],
                                   train_target[(i + 1) * num_val_samples:]],
                                  axis=0)
    model = build_model()
    history = model.fit(partial_train, partial_target,
                        validation_data=(val_data,val_targets),
                        epochs=num_epoch, 
              batch_size=1, verbose=0)
    mae_history = history.history['val_mae']
    all_mae_history.append(mae_history)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [62]:
np.mean(all_mae_history)

2.518850192055106

In [48]:
np.mean(all_scores)

2.4600839614868164

In [58]:
# final_model
final_model = build_model()
final_model.fit(train_data, train_target, epochs=100, batch_size=1,verbose=0)
test_mse_score, test_mae_score = final_model.evaluate(test_data, test_labels)

4/4 [==============================] - 0s 2ms/step - loss: 12.0886 - mae: 2.3694


In [59]:
test_mae_score

2.369441270828247